In [ ]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 27.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.7 MB/s eta 0:00:00


In [ ]:
pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.6 MB/s eta 0:00:00


In [ ]:
pip install accelerate -U

In [ ]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00


In [ ]:
pip install numpy

In [ ]:
pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.0 MB/s eta 0:00:00


In [ ]:
from transformers import BertForQuestionAnswering, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset
import numpy as np
import string
import re
from collections import Counter

# Load model, tokenizer, and dataset
model = BertForQuestionAnswering.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
dataset = load_dataset("medmcqa", download_mode='force_redownload')

def tokenize_batch(batch):
    questions = batch["question"]
    exps = batch["exp"]
    texts = [(q if q else "") + " [SEP] " + (e if e else "") for q, e in zip(questions, exps)]

    # Tokenize texts, but first check if they exceed max length, and truncate or skip if they do
    tokenized_batch = {
        "input_ids": [],
        "attention_mask": [],
        "token_type_ids": [],
        "start_positions": [],
        "end_positions": []
    }

    # Determine the correct answer text
    correct_answers = []
    for i, cop in enumerate(batch["cop"]):
        if cop == 1:
            correct_answers.append(batch["opa"][i])
        elif cop == 2:
            correct_answers.append(batch["opb"][i])
        elif cop == 3:
            correct_answers.append(batch["opc"][i])
        elif cop == 4:
            correct_answers.append(batch["opd"][i])

    for text, correct_answer in zip(texts, correct_answers):
        try:
            tokenized = tokenizer(text, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
            answer_tokens = tokenizer.tokenize(correct_answer)

            # Search for start and end position of answer tokens in the full token list
            tokens = tokenizer.tokenize(text)
            for i in range(len(tokens) - len(answer_tokens) + 1):
                if tokens[i:i+len(answer_tokens)] == answer_tokens:
                    start_position = i
                    end_position = i + len(answer_tokens) - 1
                    break
            else:
                start_position = 0
                end_position = len(answer_tokens) - 1

            tokenized_batch["input_ids"].append(tokenized["input_ids"].squeeze().tolist())
            tokenized_batch["attention_mask"].append(tokenized["attention_mask"].squeeze().tolist())
            tokenized_batch["token_type_ids"].append(tokenized["token_type_ids"].squeeze().tolist())
            tokenized_batch["start_positions"].append(start_position)
            tokenized_batch["end_positions"].append(end_position)
        except:
            # Skipping problematic examples.
            continue

    # If, for some reason, we don't get the right batch size, let's pad the results.
    while len(tokenized_batch["input_ids"]) < len(texts):
        tokenized_batch["input_ids"].append([0] * 512)
        tokenized_batch["attention_mask"].append([0] * 512)
        tokenized_batch["token_type_ids"].append([0] * 512)
        tokenized_batch["start_positions"].append(0)
        tokenized_batch["end_positions"].append(0)

    return tokenized_batch


tokenized_datasets = dataset.map(tokenize_batch, batched=True)

def normalize_answer(s):
    """Lower text and remove punctuation, articles, and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if not prediction_tokens or not ground_truth_tokens:
        return int(prediction_tokens == ground_truth_tokens)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

def exact_match_score(prediction, ground_truth):
    return int(normalize_answer(prediction) == normalize_answer(ground_truth))

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    start_logits, end_logits = predictions
    predicted_start_positions = start_logits.argmax(axis=-1)
    predicted_end_positions = end_logits.argmax(axis=-1)
    correct_start_positions = [x['start_positions'] for x in eval_dataset]
    correct_end_positions = [x['end_positions'] for x in eval_dataset]

    predicted_texts = []
    correct_texts = []

    for i, input_ids in enumerate(eval_dataset["input_ids"]):
        start_position = max(0, min(len(input_ids) - 1, predicted_start_positions[i]))
        end_position = max(0, min(len(input_ids) - 1, predicted_end_positions[i]))
        predicted_texts.append(tokenizer.decode(input_ids[start_position:end_position+1]))

        correct_start = max(0, min(len(input_ids) - 1, correct_start_positions[i]))
        correct_end = max(0, min(len(input_ids) - 1, correct_end_positions[i]))
        correct_texts.append(tokenizer.decode(input_ids[correct_start:correct_end+1]))

    exact_match = sum([1 if pred == correct else 0 for pred, correct in zip(predicted_texts, correct_texts)])
    exact_match_ratio = exact_match / len(correct_texts)

    # Compute average F1 score for the batch
    f1_scores = [f1_score(pred, correct) for pred, correct in zip(predicted_texts, correct_texts)]
    avg_f1_score = sum(f1_scores) / len(correct_texts)

    return {"exact_match_ratio": exact_match_ratio, "f1_score": avg_f1_score}




train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["test"]

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()


In [ ]:
pip install peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 1.9 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from transformers import BertForQuestionAnswering, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset
import numpy as np
import string
import re
from peft import get_peft_model, LoraConfig, TaskType
from collections import Counter

# Load model, tokenizer, and dataset
model = BertForQuestionAnswering.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
dataset = load_dataset("medmcqa", download_mode='force_redownload')


lora_config = LoraConfig(
    task_type=TaskType.QUESTION_ANS,  # Your task type
    r=8,  # LoRa attention dimension; you may want to fine-tune this based on your task
    lora_alpha=8,  # Scaling factor; set equal to r to disable scaling
    lora_dropout=0.1  # Dropout for regularization
)

model = get_peft_model(model, lora_config)

def tokenize_batch(batch):
    questions = batch["question"]
    exps = batch["exp"]
    texts = [(q if q else "") + " [SEP] " + (e if e else "") for q, e in zip(questions, exps)]

    # Tokenize texts, but first check if they exceed max length, and truncate or skip if they do
    tokenized_batch = {
        "input_ids": [],
        "attention_mask": [],
        "token_type_ids": [],
        "start_positions": [],
        "end_positions": []
    }

    # Determine the correct answer text
    correct_answers = []
    for i, cop in enumerate(batch["cop"]):
        if cop == 1:
            correct_answers.append(batch["opa"][i])
        elif cop == 2:
            correct_answers.append(batch["opb"][i])
        elif cop == 3:
            correct_answers.append(batch["opc"][i])
        elif cop == 4:
            correct_answers.append(batch["opd"][i])

    for text, correct_answer in zip(texts, correct_answers):
        try:
            tokenized = tokenizer(text, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
            answer_tokens = tokenizer.tokenize(correct_answer)

            # Search for start and end position of answer tokens in the full token list
            tokens = tokenizer.tokenize(text)
            for i in range(len(tokens) - len(answer_tokens) + 1):
                if tokens[i:i+len(answer_tokens)] == answer_tokens:
                    start_position = i
                    end_position = i + len(answer_tokens) - 1
                    break
            else:
                start_position = 0
                end_position = len(answer_tokens) - 1

            tokenized_batch["input_ids"].append(tokenized["input_ids"].squeeze().tolist())
            tokenized_batch["attention_mask"].append(tokenized["attention_mask"].squeeze().tolist())
            tokenized_batch["token_type_ids"].append(tokenized["token_type_ids"].squeeze().tolist())
            tokenized_batch["start_positions"].append(start_position)
            tokenized_batch["end_positions"].append(end_position)
        except:
            # Skipping problematic examples.
            continue

    # If, for some reason, we don't get the right batch size, let's pad the results.
    while len(tokenized_batch["input_ids"]) < len(texts):
        tokenized_batch["input_ids"].append([0] * 512)
        tokenized_batch["attention_mask"].append([0] * 512)
        tokenized_batch["token_type_ids"].append([0] * 512)
        tokenized_batch["start_positions"].append(0)
        tokenized_batch["end_positions"].append(0)

    return tokenized_batch


tokenized_datasets = dataset.map(tokenize_batch, batched=True)

def normalize_answer(s):
    """Lower text and remove punctuation, articles, and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if not prediction_tokens or not ground_truth_tokens:
        return int(prediction_tokens == ground_truth_tokens)
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

def exact_match_score(prediction, ground_truth):
    return int(normalize_answer(prediction) == normalize_answer(ground_truth))

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    start_logits, end_logits = predictions
    predicted_start_positions = start_logits.argmax(axis=-1)
    predicted_end_positions = end_logits.argmax(axis=-1)
    correct_start_positions = [x['start_positions'] for x in eval_dataset]
    correct_end_positions = [x['end_positions'] for x in eval_dataset]

    predicted_texts = []
    correct_texts = []

    for i, input_ids in enumerate(eval_dataset["input_ids"]):
        start_position = max(0, min(len(input_ids) - 1, predicted_start_positions[i]))
        end_position = max(0, min(len(input_ids) - 1, predicted_end_positions[i]))
        predicted_texts.append(tokenizer.decode(input_ids[start_position:end_position+1]))

        correct_start = max(0, min(len(input_ids) - 1, correct_start_positions[i]))
        correct_end = max(0, min(len(input_ids) - 1, correct_end_positions[i]))
        correct_texts.append(tokenizer.decode(input_ids[correct_start:correct_end+1]))

    exact_match = sum([1 if pred == correct else 0 for pred, correct in zip(predicted_texts, correct_texts)])
    exact_match_ratio = exact_match / len(correct_texts)

    # Compute average F1 score for the batch
    f1_scores = [f1_score(pred, correct) for pred, correct in zip(predicted_texts, correct_texts)]
    avg_f1_score = sum(f1_scores) / len(correct_texts)

    return {"exact_match_ratio": exact_match_ratio, "f1_score": avg_f1_score}




train_dataset = tokenized_datasets["train"]
eval_dataset = tokenized_datasets["test"]

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()


Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Generating train split:   0%|          | 0/182822 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6150 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4183 [00:00<?, ? examples/s]

Map:   0%|          | 0/182822 [00:00<?, ? examples/s]

...

Map:   0%|          | 0/6150 [00:00<?, ? examples/s]

Map:   0%|          | 0/4183 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Exact Match Ratio,F1 Score
1,2.673900,5.395859,1.000000,1.000000
2,2.699500,6.097582,0.000000,0.666667


Epoch,Training Loss,Validation Loss,Exact Match Ratio,F1 Score
1,2.673900,5.395859,1.000000,1.000000
2,2.699500,6.097582,0.000000,0.666667
3,2.767100,6.202347,0.000000,0.062500


TrainOutput(global_step=68559, training_loss=2.768761171795447, metrics={'train_runtime': 14329.3853, 'train_samples_per_second': 38.276, 'train_steps_per_second': 4.785, 'total_flos': 1.4381187189475738e+17, 'train_loss': 2.768761171795447, 'epoch': 3.0})